In [3]:
import os
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import matplotlib.pyplot as plt

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

датасет скачен с kaggle.com 

https://www.kaggle.com/datasets/abhinavnayak/catsvdogs-transformed/data

In [5]:
data_path = 'train_transformed'
list_of_images = os.listdir(data_path)
num_of_images = len(list_of_images)
num_of_images

2000

1. преобразованные данные в числа
2. добавляю числа в массив и строю тензоры

In [ ]:
# 1
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 2
X = []
y = []

cl = {'cat': 0, 'dog': 1} 

for i in range(num_of_images):
    im = Image.open(f'{data_path}/{list_of_images[i]}')
    im_tensor = transform(im)
    X.append(im_tensor)

    cat = list_of_images[i][:3]
    y.append(cl[cat])

X = torch.stack([x for x in X])
y = torch.tensor(y)
print(X.shape)
print(y.shape)

torch.Size([2000, 3, 224, 224])
torch.Size([2000])


класс датасета

In [14]:
class CatDogDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

делаем датасет, разбиваем на трейн тест, и делаем батчи

In [ ]:
dataset = CatDogDataset(X, y)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

torch.Size([32, 3, 224, 224]) torch.Size([32])


алекс нет

In [16]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


обучение сети

In [17]:
from torch.optim import Adam

model = AlexNet()

loss_func = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0001)

In [18]:
num_epochs = 5  # можешь увеличить

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {running_loss/len(train_loader):.4f} | Accuracy: {train_acc:.2f}%")

# --- Проверка на тесте ---
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Epoch [1/5] | Loss: 0.6915 | Accuracy: 51.25%
Epoch [2/5] | Loss: 0.6752 | Accuracy: 57.12%
Epoch [3/5] | Loss: 0.6582 | Accuracy: 63.44%
Epoch [4/5] | Loss: 0.6207 | Accuracy: 67.19%
Epoch [5/5] | Loss: 0.6176 | Accuracy: 66.75%
Test Accuracy: 67.00%


# Итог

акураси составило 67 процентов, что не очень хорошо, однако для такой небольшой выборки впринципе неплохо